In [8]:
# This assumes TOB-STT server is running with the rest channel enabled
# https://forum.rasa.com/t/how-to-do-conversation-with-rasa-bot-using-api/23014

import pandas as pd
import requests
import uuid


In [7]:
# Token was defined into docker-compose.yml file
tob_url = "http://tobserver:5005/webhooks/rest/webhook?token=jupytertob"

def build_request_body(session_id, user_id, question):
    return {"sender": f"{session_id}:{user_id}", "message": question}

# https://requests.readthedocs.io/en/latest/api/#requests.post
def ask_tob_v2(session_id, user_id, question):
    response = requests.post(tob_url, json = build_request_body(session_id, user_id, question))

    if response.status_code == 200:
        tob_answers = response.json()
        return tob_answers[0]['text']


In [11]:
# load v1
tob_v1 = pd.read_csv("./tb_v1_dataset_classified.csv")

tob_v1.head()

,Unnamed: 0,user,question,answer,question_class
0,0,100,hi,"Hello, how are you?",Greet
1,1,100,Simone,"Sorry, I still can't answer that.. Could you a...",Statement
2,2,100,I am Simone,"Sorry, I still can't answer that.. Could you a...",Statement
3,3,100,what is software testing,Software testing is an activity of software en...,whQuestion
4,4,100,what is structural testing,Structural testing (or White-Box testing) is a...,whQuestion


In [18]:
# runs a tob v2 session

def ask_questions(dataset):
    session_id = uuid.uuid4()
    print(session_id)
    return dataset[["user", "question"]].apply(lambda row: ask_tob_v2(session_id, row.user, row.question), axis = 1)

tob_v2 = tob_v1.copy()

tob_v2["model-20230626-203321"] = ask_questions(tob_v2)

tob_v2.head()

e9840441-bee0-4af2-8eda-ccf1d70e01ad


,Unnamed: 0,user,question,answer,question_class,model-20230626-203321
0,0,100,hi,"Hello, how are you?",Greet,Hi
1,1,100,Simone,"Sorry, I still can't answer that.. Could you a...",Statement,Hi
2,2,100,I am Simone,"Sorry, I still can't answer that.. Could you a...",Statement,Hi
3,3,100,what is software testing,Software testing is an activity of software en...,whQuestion,Software testing is an activity of software en...
4,4,100,what is structural testing,Structural testing (or White-Box testing) is a...,whQuestion,Structural testing (or White-Box testing) is a...


In [17]:
# reorganize into a new dataframe

tob_v2 = tob_v2.rename(columns={"answer": "v1_answer", "model-20230626-203321": "v2:model-20230626-203321"})

tob_final = tob_v2[["user", "question", "question_class", "v1_answer", "v2:model-20230626-203321"]]

tob_final.to_csv("./tob_final.csv")